In [1]:
import os
import sys

# Add the repository root to the Python path
repo_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, repo_root)

# Set the working directory to the repository root
os.chdir(repo_root)

In [28]:
# Prerequisites
import json
import os
from eye_ai.eye_ai import EyeAI

import pandas as pd
from pathlib import Path, PurePath
import logging

from deriva.core.ermrest_model import (
    builtin_types, 
    Schema, 
    Table, 
    Column, 
    Key, 
    ForeignKey
)
from deriva_ml import DatasetSpec, ExecutionConfiguration, DerivaML, Workflow, VersionPart
from deriva_ml import MLVocab as vc
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
# Login
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
# host = 'dev.eye-ai.org'
host = 'www.eye-ai.org'
catalog_id = "eye-ai"

gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [19]:
cache_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
working_dir = '/Users/vivi/Desktop/eye_ai/execution_cache'
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
workflow_instance = Workflow(
    name="Create Condition_Label feature",
    workflow_type="Feature_Creation",
    url='https://github.com/informatics-isi-edu/eye-ai-exec/blob/main/notebooks/feature/create_chart_label.ipynb',
    is_notebook=True
)

config = ExecutionConfiguration(
    workflow= workflow_instance, # dev'5-SG9W'
    description="Create Condition_Label for multimodal data")

# Initialize execution
execution = EA.create_execution(config)

In [ ]:
print(execution)

# Create Feature

In [8]:
image_side_table = EA.model.schemas['eye-ai'].tables['Image_Side']
condition_label_table = EA.model.schemas['eye-ai'].tables['Condition_Label']
severity_label_table = EA.model.schemas['eye-ai'].tables['Severity_Label']


In [ ]:
# severity_cv = EA.create_vocabulary(vocab_name='Chart_Label', schema='eye-ai')
chart_label_provider = Column.define(
    'Chart_Label_Provider',
    builtin_types.text,
    nullok=True
    )

severity_feature = EA.create_feature(target_table='Subject', feature_name='Chart_Label',
                                     terms=[condition_label_table, severity_label_table, image_side_table],
                                     metadata=[chart_label_provider])


# Compute Feature
## Retrived data

In [12]:
chart_label = pd.read_excel('/Users/vivi/Desktop/eye_ai/data_collection/chart_diagnosis_output-05-28-2025_NP.xlsx')
chart_label['chart_label_provider'] = chart_label['chart_label_provider'].astype(str)

chart_label['chart_label'] = chart_label['chart_label'].replace('Other Glaucoma', 'Other')
chart_label['severity_chart_label'] = chart_label['severity_chart_label'].replace('Indeterminate', 'Unspecified/Indeterminate')
chart_label['severity_chart_label'] = chart_label['severity_chart_label'].replace('Suspect', 'GS')
# chart_label

# Feature ingestion

In [ ]:
feature_name = 'Chart_Label'
Feature = EA.feature_record_class('Subject', feature_name)
Feature

In [ ]:
from IPython.display import Markdown, display
display(
    Markdown('### Feature Name'),
    [ f'Name: {c.name}, Required: {not c.nullok}' for c in Feature.feature.feature_columns]
)

In [15]:
chart_label_feature_list = [Feature(
    Execution=execution.execution_rid,
    Subject=row['RID_Subject'],
    Chart_Label_Provider=row['chart_label_provider'],
    Image_Side=row['Side'],
    Condition_Label=row['chart_label'],
    Severity_Label=row['severity_chart_label'],
    Feature_Name=feature_name,
    ) for index, row in chart_label.iterrows()]

In [ ]:
len(chart_label_feature_list)
# chart_label_feature_list

In [17]:
execution.add_features(chart_label_feature_list)

In [ ]:
execution.upload_execution_outputs()

In [31]:
for dataset in ['2-1S12', '2-7P5P', '2-N93J']:
    EA.increment_dataset_version(dataset_rid = dataset, 
                                 component = VersionPart.minor,
                                 description = 'Update Chart_Label feature',
                                 execution_rid = execution.execution_rid)
